In [1]:
from pyspark.sql import SparkSession

# Spark-Session starten
spark = SparkSession.builder \
    .appName("ShipRouteAnalysis") \
    .getOrCreate()

# Optional: Log-Level reduzieren
spark.sparkContext.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/14 17:01:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
import requests
from io import StringIO
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from pyspark.sql import SparkSession

# Funktion zum Herunterladen einer CSV-Datei
def download_csv(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        csv_data = StringIO(response.text)
        pandas_df = pd.read_csv(csv_data)
        return pandas_df
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return pd.DataFrame()  # Leeres DataFrame bei Fehler

# Liste der CSV-URLs
csv_urls = [
    "https://web.ais.dk/aisdata/aisdk-2024-03-01.zip",
    "https://web.ais.dk/aisdata/aisdk-2024-03-02.zip",
    # Füge weitere URLs hinzu
]

# Parallelisiertes Herunterladen mit ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:  # Passe die Anzahl der Threads an
    pandas_dfs = list(executor.map(download_csv, csv_urls))

# Entferne leere DataFrames und kombiniere die restlichen
pandas_dfs = [df for df in pandas_dfs if not df.empty]
combined_pandas_df = pd.concat(pandas_dfs, ignore_index=True)

# Spark-Session starten
spark = SparkSession.builder \
    .appName("ShipRouteAnalysis") \
    .getOrCreate()

# Spark DataFrame erstellen
df = spark.createDataFrame(combined_pandas_df)

# Überprüfen
df.printSchema()
df.show(5)